In [1]:
import os
import sys

import django

sys.path.append('../')  # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

  * Reading config from flourish.ini
Loading Data Encryption (init)...
 * loading keys from /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields
 * loading rsa.restricted.public ... Done.
 * loading rsa.restricted.private ... Done.
 * loading rsa.local.public ... Done.
 * loading rsa.local.private ... Done.
 * loading aes.local ... Done.
 * loading aes.restricted ... Done.
 * loading salt.local ... Done.
 * loading salt.restricted ... Done.
 Done loading Data Encryption (init)...


/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/django_q/conf.py:139: UserWarning: Retry and timeout are misconfigured. Set retry larger than timeout, 
        failure to do so will cause the tasks to be retriggered before completion. 
        See https://django-q.readthedocs.io/en/latest/configure.html#retry for details.
  warn(
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (HistoryManagerMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (ScheduleModelMixin) without inherit=True
  warnings.warn(msg, UserWarning)
/Users/nimrodmunatsi/.venv/flourish/lib/python3.9/site-packages/simple_history/models.py:164: UserWarning: HistoricalRecords added to abstract model (VisitModelMixin) without inherit=True
  warnings.wa

Loading Data Encryption ...
 * found encryption keys in /Users/nimrodmunatsi/source/flourish-code-space/flourish/crypto_fields.
 * using model django_crypto_fields.crypt.
 Done loading Data Encryption.
Loading Edc Consent ...
 * checking for site consents ...
 * registered consents 'consents' from 'flourish_caregiver'
 * registered consents 'consents' from 'flourish_child'
 * registered consents 'consents' from 'pre_flourish'
 * registered consents 'consents' from 'flourish_facet'
 * flourish_caregiver.subjectconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 2 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 3 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.subjectconsent 4 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbadolconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_caregiver.tbinformedconsent 1 covering 2020-07-01 UTC to 2025-06-30 UTC
 * flourish_child.childdum

In [12]:
from tqdm import tqdm
from flourish_child.helper_classes.utils import child_utils
from edc_constants.constants import NEG, POS, YES
from flourish_caregiver.models import CaregiverChildConsent


def child_hiv_exposure(subject_identifier=None,
                       study_maternal_identifier=None,
                       caregiver_subject_identifier=None):
    child_dataset_cls = django_apps.get_model(
        'flourish_child.childdataset')

    if study_maternal_identifier:
        child_dataset_objs = child_dataset_cls.objects.filter(
            study_maternal_identifier=study_maternal_identifier)

        if child_dataset_objs:
            if child_dataset_objs[0].infant_hiv_exposed in ['Exposed', 'exposed']:
                return 'HEU'
            elif child_dataset_objs[0].infant_hiv_exposed in ['Unexposed',
                                                              'unexposed']:
                return 'HUU'
    else:
        rapid_test_cls = django_apps.get_model(
            'flourish_caregiver.hivrapidtestcounseling')
        maternal_hiv_status = None

        try:
            rapid_test_obj = rapid_test_cls.objects.get(
                maternal_visit__visit_code='1000M',
                maternal_visit__visit_code_sequence=0,
                maternal_visit__subject_identifier=caregiver_subject_identifier,
                rapid_test_done=YES)
        except rapid_test_cls.DoesNotExist:
            antenatal_enrollment_cls = django_apps.get_model(
                'flourish_caregiver.antenatalenrollment')
            try:
                antenatal_enrollment = antenatal_enrollment_cls.objects.get(
                    subject_identifier=caregiver_subject_identifier,
                    child_subject_identifier=subject_identifier)
            except antenatal_enrollment_cls.DoesNotExist:
                # To refactor to include new enrollees
                maternal_hiv_status = 'UNK'
            else:
                maternal_hiv_status = antenatal_enrollment.enrollment_hiv_status
        else:
            maternal_hiv_status = rapid_test_obj.result

        if maternal_hiv_status == POS:
            return 'HEU'
        elif maternal_hiv_status == NEG:
            return 'HUU'
        else:
            return 'UNK'


def screening_identifier(subject_identifier=None):
    """Returns a screening identifier.
    """
    consent_cls = django_apps.get_model(
        'flourish_caregiver.subjectconsent')
    consent = consent_cls.objects.filter(
        subject_identifier=subject_identifier)
    if consent:
        return consent.last().screening_identifier
    return None


def previous_bhp_study(subject_identifier=None):
    caregiver_child_consent_cls = django_apps.get_model(
        'flourish_caregiver.caregiverchildconsent')
    if subject_identifier:
        try:
            caregiver_child_consent_obj = caregiver_child_consent_cls.objects.filter(
                subject_identifier=subject_identifier).latest('consent_datetime')
        except caregiver_child_consent_cls.DoesNotExist:
            return None
        else:
            return caregiver_child_consent_obj.get_protocol


def subject_screning_obj(screening_identifier):
    subject_screening_class = django_apps.get_model(
        'flourish_caregiver.screeningpriorbhpparticipants'
    )
    try:
        return subject_screening_class.objects.get(
            screening_identifier=screening_identifier)
    except subject_screening_class.DoesNotExist:
        return None


def study_maternal_identifier(screening_identifier=None):
    dataset_cls = django_apps.get_model(
        'flourish_caregiver.maternaldataset')
    if screening_identifier:
        try:
            dataset_obj = dataset_cls.objects.get(
                screening_identifier=screening_identifier)
        except dataset_cls.DoesNotExist:
            return subject_screning_obj(
                screening_identifier).study_maternal_identifier if (
                subject_screning_obj(screening_identifier)) else None
        else:
            return dataset_obj.study_maternal_identifier

kids = list(
    set(CaregiverChildConsent.objects.all().values_list('subject_identifier', flat=True)))
unk_hiv_exp = []
for kid in tqdm(kids):
    caregiver_sid = child_utils.caregiver_subject_identifier(
        subject_identifier=kid)
    screening_id = screening_identifier(
        subject_identifier=caregiver_sid)
    previous_study = previous_bhp_study(
        subject_identifier=kid)
    study_maternal_id= study_maternal_identifier(
        screening_identifier=screening_id)
    child_exposure_status = child_hiv_exposure(
        subject_identifier=kid,
        study_maternal_identifier=study_maternal_id,
        caregiver_subject_identifier=caregiver_sid)
    
    if child_exposure_status == 'UNK':
        unk_hiv_exp.append({
            'sid':kid,
            'screening_id':screening_id,
            'caregiver_sid':caregiver_sid,
            'previous_study':previous_study,
            'study_maternal_id':study_maternal_id,
        })
        
len(unk_hiv_exp)
    

100%|██████████| 1304/1304 [02:59<00:00,  7.27it/s]


38

In [13]:
unk_hiv_exp

[{'sid': 'B142-040990371-2-60',
  'screening_id': None,
  'caregiver_sid': None,
  'previous_study': None,
  'study_maternal_id': None},
 {'sid': 'B142-040990472-8-60',
  'screening_id': None,
  'caregiver_sid': None,
  'previous_study': None,
  'study_maternal_id': None},
 {'sid': 'B142-040990664-0-10',
  'screening_id': None,
  'caregiver_sid': None,
  'previous_study': None,
  'study_maternal_id': None},
 {'sid': 'B142-040990920-6-10',
  'screening_id': None,
  'caregiver_sid': None,
  'previous_study': None,
  'study_maternal_id': None},
 {'sid': 'B142-040991217-6-10',
  'screening_id': None,
  'caregiver_sid': None,
  'previous_study': None,
  'study_maternal_id': None},
 {'sid': 'B142-040991159-0-10',
  'screening_id': None,
  'caregiver_sid': None,
  'previous_study': None,
  'study_maternal_id': None},
 {'sid': 'B142-040991215-0-10',
  'screening_id': None,
  'caregiver_sid': None,
  'previous_study': None,
  'study_maternal_id': None},
 {'sid': 'B142-040991264-8-10',
  'screen

In [11]:
from flourish_child.models import ChildDataset
from flourish_caregiver.models import CaregiverLocator, ScreeningPriorBhpParticipants, MaternalDataset

for obj in unk_hiv_exp:
    if obj.get('screening_id'):
        try:
            screening_obj=ScreeningPriorBhpParticipants.objects.get(
                screening_identifier=obj.get('screening_id')
            )
        except ScreeningPriorBhpParticipants.DoesNotExist:
            child_consent=CaregiverChildConsent.objects.filter(subject_identifier=obj.get('sid')).latest('consent_datetime')
            child_dataset_obj=ChildDataset.objects.get(study_child_identifier=child_consent.study_child_identifier)
            screening_obj=ScreeningPriorBhpParticipants.objects.get(study_maternal_identifier=child_dataset_obj.study_maternal_identifier)
            if obj.get('screening_id') != screening_obj.screening_identifier:
                screening_obj.screening_identifier = obj.get('screening_id')
                screening_obj.save()
            maternal_dataset_obj = MaternalDataset.objects.get(study_maternal_identifier=child_dataset_obj.study_maternal_identifier)
            if obj.get('screening_id') != maternal_dataset_obj.screening_identifier: 
                maternal_dataset_obj.screening_identifier= obj.get('screening_id')
                maternal_dataset_obj.save()
            locator_obj = CaregiverLocator.objects.get(study_maternal_identifier=child_dataset_obj.study_maternal_identifier)
            if obj.get('screening_id') != locator_obj.screening_identifier:
                locator_obj.subject_identifier = obj.get('screening_id')
                locator_obj.save()
            
            

{'_state': <django.db.models.base.ModelState at 0x17395ff70>,
 'created': datetime.datetime(2021, 4, 24, 3, 30, 30, 908741, tzinfo=<UTC>),
 'modified': datetime.datetime(2021, 4, 24, 3, 30, 30, 908775, tzinfo=<UTC>),
 'user_created': '',
 'user_modified': '',
 'hostname_created': 'flourish',
 'hostname_modified': 'flourish',
 'revision': '',
 'device_created': '99',
 'device_modified': '99',
 'id': UUID('0010d2ba-831f-4d27-93a8-26e4e1998695'),
 'site_id': 40,
 'subject_identifier': '',
 'slug': '|s99tv8rp|056-3981240-0',
 'screening_identifier': 'S99TV8RP',
 'study_maternal_identifier': '056-3981240-0',
 'first_name': None,
 'last_name': None,
 'protocol': 'Mpepu',
 'delivdt': datetime.date(2012, 8, 28),
 'site_name': 'Lobatse',
 'mom_enrolldate': datetime.date(2012, 8, 28),
 'delivmeth': '',
 'delivery_location': 'Lobatse',
 'ega_delivery': None,
 'mom_age_enrollment': '18-24 years',
 'mom_hivstatus': 'HIV-infected',
 'parity': 1,
 'gravida': None,
 'mom_education': 'Secondary',
 'mom